<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/Falcon_1b_with_Biomed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers
!pip install evaluate numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-nom1pdk3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-nom1pdk3
  Resolved https://github.com/huggingface/accelerate to commit c7c99a30ea88f00c854017d362e72b9cf60c7db6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-no

# Load the dataset

In [ ]:
from datasets import load_dataset

dataset_name = "covid_qa_deepset"
dataset = load_dataset(dataset_name)
dataset = dataset["train"].train_test_split(test_size=200, train_size=1413)
dataset = dataset.filter(lambda example: not "answer_category" in example['answers'])

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1413 [00:00<?, ? examples/s]

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 200
    })
})

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AutoTokenizer

# load distilled bert tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
#tokenizer.pad_token = tokenizer.eos_token


In [ ]:
tokenizer("I am Son")

{'input_ids': [101, 1045, 2572, 2365, 102], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:

def preprocess_training(example):
    inputs = tokenizer(
        example['question'],
        example["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = example["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_eval(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

training_dataset = dataset['train']
training_dataset = training_dataset.map(preprocess_training, batched=True, remove_columns=dataset['train'].column_names)

eval_dataset = dataset['test']
eval_dataset = eval_dataset.map(preprocess_eval, batched=True, remove_columns=dataset['test'].column_names)


Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
training_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1413
})

In [ ]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 3954
})

In [ ]:
training_dataset['start_positions']


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_dataset.push_to_hub("goodcoffee/covidQA_training_v2")
eval_dataset.push_to_hub("goodcoffee/covidQA_eval_v2")


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

# Load the model

In [ ]:
import torch
import transformers
from transformers import AutoModelForQuestionAnswering, BitsAndBytesConfig, BertForQuestionAnswering, AutoConfig

model_name = "distilbert-base-uncased-distilled-squad"


model = DistilBertForQuestionAnswering.from_pretrained(
    model_name,
)
model.config.use_cache = False


# Load evaluation metrics

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
metric.description

'\nThe F1 score is the harmonic mean of the precision and recall. It can be computed with the equation:\nF1 = 2 * (precision * recall) / (precision + recall)\n'

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy = "steps",
    save_steps = 10,
    logging_steps = 10,
    max_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


In [ ]:
from transformers import Trainer

max_seq_length = 2048

trainer = Trainer(
    model=model,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator = data_collator
)

In [ ]:
training_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1413
})

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key="2ba90e109c0da4331467290174e54e3483f0d494")
wandb.init(project="QLoRA covidQA")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bobbyngoson204 (intelligentsystem). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()


Step,Training Loss
10,0.916600
20,0.781100
30,0.843000
40,0.636900
50,0.576800
60,0.629800
70,0.749700
80,0.599200
90,0.623800
100,0.448500


TrainOutput(global_step=500, training_loss=0.34092417216300963, metrics={'train_runtime': 6623.9984, 'train_samples_per_second': 1.208, 'train_steps_per_second': 0.075, 'total_flos': 1038038879754240.0, 'train_loss': 0.34092417216300963, 'epoch': 5.62})

#Run evaluation

In [ ]:
trainer.evaluate()

{'eval_runtime': 973.7854,
 'eval_samples_per_second': 4.06,
 'eval_steps_per_second': 0.255}

# Inference

In [ ]:
# Before training
from transformers import pipeline

question = dataset['train']['question'][5]
context = dataset['train']['context'][5]
print(question)

pipe = QuestionAnsweringPipeline(model="distilbert-base-uncased-distilled-squad")
pipe(question=question, context=context, max_answer_len=300, max_question_len=300)

What is also linked with the chronic inflammation that precedes the malignancies?


{'score': 0.5608590245246887,
 'start': 18519,
 'end': 18540,
 'answer': 'human papilloma virus'}

In [ ]:
#After training
from transformers import QuestionAnsweringPipeline
question = dataset['train']['question'][5]
context = dataset['train']['context'][5]
print(question)
#print(context)

pipeline = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)
print(pipeline(question=question, context=context, max_answer_len=300, max_question_len=300))

What is also linked with the chronic inflammation that precedes the malignancies?
{'score': 0.8584696650505066, 'start': 18519, 'end': 18546, 'answer': 'human papilloma virus (HPV)'}
